In [1]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://localhost:5432/mydatabase')

In [2]:
transactions = pd.read_csv('../transaction.csv', sep=';')
transactions['list_price'] = transactions['list_price'].str.replace(',', '.').astype('float')
transactions['standard_cost'] = transactions['standard_cost'].str.replace(',', '.').astype('float')
transactions['transaction_date'] = transactions['transaction_date'].apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [3]:
transactions.to_sql('transactions', engine, if_exists='replace', index=False)

1000

In [4]:
customer = pd.read_csv('../customer.csv', sep=';')
customer.to_sql('customer', engine, if_exists='replace', index=False)
customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [6]:
query = """
SELECT 
    job_industry_category, 
    COUNT(customer_id) AS customer_count
FROM customer
GROUP BY job_industry_category
ORDER BY customer_count DESC;

"""

pd.read_sql_query(query,con=engine)

,job_industry_category,customer_count
0,Manufacturing,799
1,Financial Services,774
2,None,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [10]:
query = """
SELECT 
    TO_CHAR(transaction_date, 'YYYY-MM') AS transaction_month,
    c.job_industry_category,
    SUM(t.list_price) AS total_transactions
FROM transactions t
JOIN customer c ON t.customer_id = c.customer_id
GROUP BY transaction_month, c.job_industry_category
ORDER BY transaction_month, c.job_industry_category;

"""

pd.read_sql_query(query,con=engine)

,transaction_month,job_industry_category,total_transactions
0,2017-01,Argiculture,43513.82
1,2017-01,Entertainment,64089.92
2,2017-01,Financial Services,366383.71
3,2017-01,Health,286860.38
4,2017-01,IT,107783.37
...,...,...,...
115,2017-12,Manufacturing,319248.23
116,2017-12,Property,117263.20
117,2017-12,Retail,153369.72
118,2017-12,Telecommunications,25852.57


Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [14]:
query = """
SELECT 
    t.brand, 
    COUNT(t.transaction_id) AS online_order_count
FROM transactions t
JOIN customer c ON t.customer_id = c.customer_id
WHERE t.online_order = 'Yes' 
    AND t.order_status = 'Approved'
    AND c.job_industry_category = 'IT'
    AND brand IS NOT NULL
GROUP BY t.brand

"""

pd.read_sql_query(query, con=engine)

,brand,online_order_count
0,OHM Cycles,78
1,Trek Bicycles,82
2,WeareA2B,90
3,Solex,101
4,Norco Bicycles,92
5,Giant Bicycles,89


Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

In [16]:
query = """
SELECT 
    c.customer_id,
    SUM(t.list_price) AS total_spent,
    MAX(t.list_price) AS max_transaction,
    MIN(t.list_price) AS min_transaction,
    COUNT(t.transaction_id) AS transaction_count
FROM transactions t
JOIN customer c ON t.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC, transaction_count DESC;

"""

pd.read_sql_query(query,con=engine)

,customer_id,total_spent,max_transaction,min_transaction,transaction_count
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,1597,18052.68,2091.47,360.40,12
3,941,17898.46,2091.47,1057.51,10
4,2788,17258.94,2083.94,183.86,11
...,...,...,...,...,...
3488,2423,202.62,202.62,202.62,1
3489,3189,200.70,100.35,100.35,2
3490,2274,142.98,71.49,71.49,2
3491,2532,71.49,71.49,71.49,1


In [18]:
query = """
SELECT DISTINCT 
    c.customer_id,
    SUM(t.list_price) OVER (PARTITION BY c.customer_id) AS total_spent,
    MAX(t.list_price) OVER (PARTITION BY c.customer_id) AS max_transaction,
    MIN(t.list_price) OVER (PARTITION BY c.customer_id) AS min_transaction,
    COUNT(t.transaction_id) OVER (PARTITION BY c.customer_id) AS transaction_count
FROM transactions t
JOIN customer c ON t.customer_id = c.customer_id
ORDER BY total_spent DESC, transaction_count DESC;

"""

pd.read_sql_query(query,con=engine)

,customer_id,total_spent,max_transaction,min_transaction,transaction_count
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,1597,18052.68,2091.47,360.40,12
3,941,17898.46,2091.47,1057.51,10
4,2788,17258.94,2083.94,183.86,11
...,...,...,...,...,...
3488,2423,202.62,202.62,202.62,1
3489,3189,200.70,100.35,100.35,2
3490,2274,142.98,71.49,71.49,2
3491,2532,71.49,71.49,71.49,1


Результаты одинаковые

Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [22]:
query = """
WITH t AS (
    SELECT
        c.first_name,
        c.last_name,
        SUM(t.list_price) AS total_transaction_sum
    FROM customer c
    JOIN  transactions t ON c.customer_id = t.customer_id
    WHERE t.list_price IS NOT NULL
    GROUP BY
        c.customer_id, c.first_name, c.last_name
)
SELECT
    first_name,
    last_name,
    total_transaction_sum
FROM t
WHERE total_transaction_sum = (SELECT MIN(total_transaction_sum) FROM t);

"""

pd.read_sql_query(query,con=engine)

,first_name,last_name,total_transaction_sum
0,Hamlen,Slograve,60.34


In [ ]:
query = """
WITH t AS (
    SELECT
        c.first_name,
        c.last_name,
        SUM(t.list_price) AS total_transaction_sum
    FROM customer c
    JOIN  transactions t ON c.customer_id = t.customer_id
    WHERE t.list_price IS NOT NULL
    GROUP BY
        c.customer_id, c.first_name, c.last_name
)
SELECT
    first_name,
    last_name,
    total_transaction_sum
FROM t
WHERE total_transaction_sum = (SELECT MAX(total_transaction_sum) FROM t);

"""

pd.read_sql_query(query,con=engine)

,first_name,last_name,total_transaction_sum
0,Jillie,Fyndon,19071.32


Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [24]:
query = """
WITH FirstTransactions AS (
    SELECT
        t.transaction_id,
        t.customer_id,
        t.transaction_date,
        ROW_NUMBER() OVER (PARTITION BY t.customer_id ORDER BY t.transaction_date) AS rn
    FROM transactions t
)
SELECT
    ft.transaction_id,
    ft.customer_id,
    ft.transaction_date
FROM FirstTransactions ft
WHERE ft.rn = 1;
"""

pd.read_sql_query(query,con=engine)

,transaction_id,customer_id,transaction_date
0,9785,1,2017-01-05
1,2261,2,2017-05-04
2,10302,3,2017-02-23
3,12441,4,2017-04-03
4,2291,5,2017-03-03
...,...,...,...
3489,8276,3497,2017-09-01
3490,13469,3498,2017-02-03
3491,2794,3499,2017-01-12
3492,6309,3500,2017-01-09


Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [ ]:
query = """
WITH TransactionIntervals AS (
    SELECT
        c.first_name,
        c.last_name,
        c.job_title,
        t.transaction_date,
        (t.transaction_date::date - LAG(t.transaction_date::date) OVER (PARTITION BY t.customer_id ORDER BY t.transaction_date)) AS interval_days
    FROM customer c
    JOIN transactions t ON c.customer_id = t.customer_id
),
MaxInterval AS (
    SELECT
        first_name,
        last_name,
        job_title,
        MAX(interval_days) AS max_interval
    FROM TransactionIntervals
    GROUP BY first_name, last_name, job_title
)
SELECT
    first_name,
    last_name,
    job_title,
    max_interval
FROM MaxInterval
WHERE max_interval = (SELECT MAX(max_interval) FROM MaxInterval);

"""

pd.read_sql_query(query,con=engine)

,first_name,last_name,job_title,max_interval
0,Susanetta,None,Legal Assistant,357
